In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import csv
import cv2
import tqdm.notebook as tq
from sklearn.utils import shuffle
from keras.optimizers import *
from keras.losses import *
from keras import Sequential, Model
from keras.layers import *
import PIL
import PIL.Image
import keras.backend as K
import pickle
import keras
from keras.models import load_model
import glob
import os
import ntpath
#tf.config.run_functions_eagerly(True)

In [2]:
data_location  = 'Dataset/carla_dataset/'
model_location = 'Models/'

In [3]:
class CarSimulatorAdv(tf.keras.Model):

    def __init__(self):
        super(CarSimulatorAdv, self).__init__()
        self.classifier = load_model(model_location+'model.h5')
        with open(model_location+'mean_image_rgb.pickle', 'rb') as f:
            self.mean = pickle.load(f, encoding='latin1')  # dictionary type
        self.objectdetector = cv2.dnn.readNetFromDarknet(model_location+'yolov3-spp.cfg', model_location+'yolov3-spp.weights')
        self.objectdetector.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        self.objectdetector.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL_FP16)
        self.layers_all = self.objectdetector.getLayerNames()
        self.layers_names_output = [self.layers_all[i[0] - 1] for i in self.objectdetector.getUnconnectedOutLayers()]
        self.probability_minimum = 0.2
        self.threshold = 0.2

    def call(self, input_image, training=False):
        image_BGR = cv2.imread(input_image)
        h, w = image_BGR.shape[:2]

        blob = cv2.dnn.blobFromImage(image_BGR, 1 / 255.0, (416, 416), swapRB=True, crop=False)
        self.objectdetector.setInput(blob)
        output_from_network = self.objectdetector.forward(self.layers_names_output)

        bounding_boxes = []
        confidences = []

        for result in output_from_network:
            for detected_objects in result:
                scores = detected_objects[5:]
                class_current = np.argmax(scores)
                confidence_current = scores[class_current]

                if confidence_current > self.probability_minimum:
                    box_current = detected_objects[0:4] * np.array([w, h, w, h])
                    x_center, y_center, box_width, box_height = box_current
                    x_min = int(x_center - (box_width / 2))
                    y_min = int(y_center - (box_height / 2))
                    bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                    confidences.append(float(confidence_current))

        results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, self.probability_minimum, self.threshold)

        detected_sign = []
        if len(results) > 0:
            for i in results.flatten():
                x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
                box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]
                c_ts = image_BGR[y_min:y_min+int(box_height), x_min:x_min+int(box_width), :]
                if c_ts.shape[:1] == (0,) or c_ts.shape[1:2] == (0,):
                    pass
                else:
                    blob_ts = cv2.dnn.blobFromImage(c_ts, 1 / 255.0, size=(32, 32), swapRB=True, crop=False)
                    blob_ts[0] = blob_ts[0, :, :, :] - self.mean['mean_image_rgb']
                    blob_ts = blob_ts.transpose(0, 2, 3, 1)

                    if input_image[-4:] == '.png':
                        input_image = input_image.replace('.png', '.jpg')
                    tf.keras.preprocessing.image.save_img(input_image.replace('.jpg', '_detect.png'), blob_ts[0], scale=True)
                    bolb_img = tf.keras.preprocessing.image.load_img(input_image.replace('.jpg', '_detect.png'), grayscale=False, color_mode='rgb', target_size=(30, 30, 3), interpolation='nearest')
                    bolb_ts = keras.preprocessing.image.img_to_array(bolb_img)
                    X_test = np.array([bolb_ts])
                    scores = np.argmax(self.classifier.predict(X_test), axis=-1)
                    #print(self.classifier.predict(X_test))
                    if scores[0] in range(3):
                        box_info = (x_min, y_min, x_min+box_width, y_min+box_height)
                        detected_sign.append((X_test, scores[0], box_info))
                        break
        return detected_sign

In [4]:
def image_mapper_adv(input_image, sign, box_info):
    image_path = ntpath.basename(input_image)
    image_b = tf.keras.preprocessing.image.load_img(input_image, grayscale=False, color_mode='rgb', interpolation='nearest')
    imagebg = keras.preprocessing.image.img_to_array(image_b)
    imagebg = np.array(imagebg)

    x1, y1, x2, y2 = box_info
    noisy = tf.keras.preprocessing.image.load_img(sign, grayscale=False, color_mode='rgb', target_size=(y2-y1, x2-x1), interpolation='nearest')
    X_noisy = keras.preprocessing.image.img_to_array(noisy)
    X_noisy = np.array(X_noisy)

    imagebg[y1:y2, x1:x2] = X_noisy

    tf.keras.preprocessing.image.save_img(input_image.replace(image_path, 'AdvImages/'+image_path.replace('.jpg', '_noisy.png')), imagebg, scale=True)

In [5]:
with open(data_location + 'carla_dataset_annotation.csv', 'r') as f:
    lines = f.read().splitlines()
    scores = {}
    for l in lines:
        x = l.split(',')
        scores[x[0]] = x[1]

In [6]:
modelAdv = CarSimulatorAdv()
generator = load_model('saved_model2/gen_final.h5')

In [7]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
def generate_adv():
    count = 0
    img_count = 0
    index = []
    detect_count = 0
    for image in glob.glob(os.path.join(data_location, '*.jpg')):
        #print(img_count)
        try:
            detect = modelAdv(image)
            if len(detect)>0:
                detect_count+=1
                image_path = ntpath.basename(image)
                if int(scores[image_path]) == detect[-1][1]:
                    sign_img = normalization_layer(detect[-1][0])
                    noisy = generator(sign_img)
                    noisy_path = data_location + 'NoisySigns/'+image_path.replace('.jpg', '_noisysign.png')
                    tf.keras.preprocessing.image.save_img(noisy_path, noisy[0], scale=True)
                    image_mapper_adv(image, noisy_path, detect[-1][2])
                    adv_detect = modelAdv(data_location +'AdvImages/'+image_path.replace('.jpg','_noisy.png'))
                    if len(adv_detect)>0:
                        print(image_path, detect[-1][1], adv_detect[-1][1])
                        if int(scores[image_path]) != adv_detect[-1][1]:
                            count+=1
                            index.append(image_path)
                # if img_count == 40:
                #     break
        except:
            continue
        img_count+=1
    print(detect_count)
    return count, index

In [8]:
count = generate_adv()

DATASET_0001002.jpg 0 0
DATASET_0001005.jpg 0 0
DATASET_0001011.jpg 0 2
DATASET_0001017.jpg 0 2
DATASET_000102.jpg 1 1
DATASET_0001021.jpg 0 2
DATASET_0001023.jpg 0 2
DATASET_0001027.jpg 0 2
DATASET_0001033.jpg 0 2
DATASET_0001037.jpg 0 2
DATASET_0001039.jpg 0 2
DATASET_0001041.jpg 0 2
DATASET_0001043.jpg 0 2
DATASET_0001045.jpg 0 2
DATASET_0001046.jpg 0 0
DATASET_0001049.jpg 0 2
DATASET_0001055.jpg 1 2
DATASET_000106.jpg 1 1
DATASET_0001068.jpg 0 0
DATASET_0001069.jpg 0 0
DATASET_0001071.jpg 1 1
DATASET_0001073.jpg 1 1
DATASET_0001075.jpg 1 1
DATASET_0001077.jpg 1 1
DATASET_0001079.jpg 1 2
DATASET_000108.jpg 1 1
DATASET_0001088.jpg 0 0
DATASET_0001089.jpg 0 0
DATASET_0001102.jpg 0 0
DATASET_000115.jpg 1 2
DATASET_0001151.jpg 0 2
DATASET_0001187.jpg 0 2
DATASET_0001199.jpg 0 0
DATASET_0001223.jpg 0 2
DATASET_000124.jpg 2 2
DATASET_0001248.jpg 0 2
DATASET_0001268.jpg 0 2
DATASET_0001270.jpg 0 2
DATASET_0001272.jpg 0 0
DATASET_0001276.jpg 0 0
DATASET_0001280.jpg 0 0
DATASET_0001284.jpg 0

DATASET_0009388.jpg 0 0
DATASET_0009389.jpg 0 2
DATASET_0009391.jpg 0 2
DATASET_0009392.jpg 0 0
DATASET_0009393.jpg 0 0
DATASET_0009394.jpg 0 0
DATASET_0009402.jpg 0 2
DATASET_000949.jpg 0 2
DATASET_000955.jpg 0 2
DATASET_000958.jpg 0 0
DATASET_00096.jpg 1 1
DATASET_000965.jpg 0 0
DATASET_000967.jpg 0 0
DATASET_000969.jpg 0 0
DATASET_000973.jpg 0 2
DATASET_000979.jpg 0 0
DATASET_000980.jpg 0 0
861


In [10]:
count[0], count[1]

(155,
 ['DATASET_0001011.jpg',
  'DATASET_0001017.jpg',
  'DATASET_0001021.jpg',
  'DATASET_0001023.jpg',
  'DATASET_0001027.jpg',
  'DATASET_0001033.jpg',
  'DATASET_0001037.jpg',
  'DATASET_0001039.jpg',
  'DATASET_0001041.jpg',
  'DATASET_0001043.jpg',
  'DATASET_0001045.jpg',
  'DATASET_0001049.jpg',
  'DATASET_0001055.jpg',
  'DATASET_0001079.jpg',
  'DATASET_000115.jpg',
  'DATASET_0001151.jpg',
  'DATASET_0001187.jpg',
  'DATASET_0001223.jpg',
  'DATASET_0001248.jpg',
  'DATASET_0001268.jpg',
  'DATASET_0001270.jpg',
  'DATASET_0001294.jpg',
  'DATASET_0001296.jpg',
  'DATASET_0001301.jpg',
  'DATASET_0001310.jpg',
  'DATASET_000158.jpg',
  'DATASET_00016.jpg',
  'DATASET_000168.jpg',
  'DATASET_00018.jpg',
  'DATASET_000181.jpg',
  'DATASET_000184.jpg',
  'DATASET_000196.jpg',
  'DATASET_00020.jpg',
  'DATASET_000204.jpg',
  'DATASET_000206.jpg',
  'DATASET_000208.jpg',
  'DATASET_000210.jpg',
  'DATASET_000214.jpg',
  'DATASET_00022.jpg',
  'DATASET_000232.jpg',
  'DATASET_000